In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
pitching_23 = pd.read_csv('../../../Desktop/Joliet Slammers/Baseball Reference/2023 Frontier League Pitching.csv')
pitching_23['Name'] = pitching_23['Name'].str.replace('*', '')
pitching_23['Name'] = pitching_23['Name'].str.replace('#', '')
pitching_23['Name'] = pitching_23['Name'].str.replace('?', '')
pitching_23['Year'] = 2023

pitching_22 = pd.read_csv('../../../Desktop/Joliet Slammers/Baseball Reference/2022 Frontier League Pitching.csv')
pitching_22['Name'] = pitching_22['Name'].str.replace('*', '')
pitching_22['Name'] = pitching_22['Name'].str.replace('#', '')
pitching_22['Year'] = 2022

pitching_21 = pd.read_csv('../../../Desktop/Joliet Slammers/Baseball Reference/2021 Frontier League Pitching.csv')
pitching_21['Name'] = pitching_21['Name'].str.replace('*', '')
pitching_21['Name'] = pitching_21['Name'].str.replace('#', '')
pitching_22['Name'] = pitching_22['Name'].str.replace('?', '')
pitching_21['Name'] = pitching_21['Name'].str.replace('?', '')
pitching_21['Year'] = 2021

hitting_23 = pd.read_csv('../../../Desktop/Joliet Slammers/Baseball Reference/2023 Frontier League Hitting.csv')
hitting_23['Name'] = hitting_23['Name'].str.replace('*', '')
hitting_23['Name'] = hitting_23['Name'].str.replace('#', '')
hitting_23['Name'] = hitting_23['Name'].str.replace('?', '')
hitting_23['Year'] = 2023

hitting_22 = pd.read_csv('../../../Desktop/Joliet Slammers/Baseball Reference/2022 Frontier League Hitting.csv')
hitting_22['Name'] = hitting_22['Name'].str.replace('*', '')
hitting_22['Name'] = hitting_22['Name'].str.replace('#', '')
hitting_22['Name'] = hitting_22['Name'].str.replace('?', '')
hitting_22['Year'] = 2022

hitting_21 = pd.read_csv('../../../Desktop/Joliet Slammers/Baseball Reference/2021 Frontier League Hitting.csv')
hitting_21['Name'] = hitting_21['Name'].str.replace('*', '')
hitting_21['Name'] = hitting_21['Name'].str.replace('#', '')
hitting_21['Name'] = hitting_21['Name'].str.replace('?', '')
hitting_21['Year'] = 2021

In [3]:
def combine_rows(df):
    return df.groupby('Name', as_index=False).agg({
        col: 'first' for col in df.columns
    })

pitching_23 = combine_rows(pitching_23)
pitching_22 = combine_rows(pitching_22)
pitching_21 = combine_rows(pitching_21)

hitting_23 = combine_rows(hitting_23)
hitting_22 = combine_rows(hitting_22)
hitting_21 = combine_rows(hitting_21)

In [4]:
hitting_23['1B'] = hitting_23['H'] - (hitting_23['2B'] + hitting_23['3B'] + hitting_23['HR'])
hitting_22['1B'] = hitting_22['H'] - (hitting_22['2B'] + hitting_22['3B'] + hitting_22['HR'])
hitting_21['1B'] = hitting_21['H'] - (hitting_21['2B'] + hitting_21['3B'] + hitting_21['HR'])

In [5]:
pitching_23 = pitching_23[pitching_23.IP > 10]
pitching_22 = pitching_22[pitching_22.IP > 10]
pitching_21 = pitching_21[pitching_21.IP > 10]

In [6]:
hitting_23 = hitting_23[hitting_23.PA > 20]
hitting_22 = hitting_22[hitting_22.PA > 20]
hitting_21 = hitting_21[hitting_21.PA > 20]

In [7]:
def calculate_rates(df):
    df['BB/In'] = df['BB'] / df['IP']
    df['K/In'] = df['SO'] / df['IP']
    df['HR/In'] = df['HR'] / df['IP']
    df['W/In'] = df['W'] / df['IP']
    df['L/In'] = df['L'] / df['IP']
    df['S/In'] = df['SV'] / df['IP']
    df['ER/In'] = df['ER'] / df['IP']
    df['R/In'] = df['R'] / df['IP']
    df['G/In'] = df['G'] / df['IP']
    df['GS/In'] = df['GS'] / df['IP']
    
    return df

pitching_21 = calculate_rates(pitching_21)
pitching_22 = calculate_rates(pitching_22)
pitching_23 = calculate_rates(pitching_23)

In [8]:
def calculate_rates_hitters(df):
    df['BB/PA'] = df['BB'] / df['PA']
    df['K/PA'] = df['SO'] / df['PA']
    df['HR/PA'] = df['HR'] / df['PA']
    df['R/PA'] = df['R'] / df['PA']
    df['1B/PA'] = df['1B'] / df['PA']
    df['2B/PA'] = df['2B'] / df['PA']
    df['3B/PA'] = df['3B'] / df['PA']
    df['SB/PA'] = df['SB'] / df['PA']
    df['AB/PA'] = df['AB'] / df['PA']
    df['SF/PA'] = df['SF'] / df['PA']
    df['CS/PA'] = df['CS'] / df['PA']
    df['HBP/PA'] = df['HBP'] / df['PA']
    
    return df

hitting_21 = calculate_rates_hitters(hitting_21)
hitting_22 = calculate_rates_hitters(hitting_22)
hitting_23 = calculate_rates_hitters(hitting_23)

In [9]:
pitching = pd.concat([pitching_23, pitching_22, pitching_21])
hitting = pd.concat([hitting_23, hitting_22, hitting_21])

In [10]:
pitching_weights = {2023: 3, 2022: 2, 2021: 1}
hitting_weights = {2023: 5, 2022: 4, 2021: 3}

In [11]:
def calculate_pitching_metrics(group):
    divisor = 0
    for year, group_data in group.groupby('Year'):
        divisor += pitching_weights[year]
        
    result = {}
    for metric in ['K/In', 'BB/In', 'HR/In', 'W/In', 'L/In', 'S/In', 'ER/In', 'R/In', 'G/In', 'GS/In']:
        result[metric] = (group[metric] * group.groupby('Year')['Year'].transform(lambda x: int(x.iloc[0]) if not isinstance(x.iloc[0], int) else x.iloc[0]).map(pitching_weights) / divisor).sum()

    return pd.Series(result)

pitcher_proj = pitching.groupby('Name').apply(calculate_pitching_metrics).reset_index()
pitcher_proj = pitcher_proj.merge(pitching.groupby('Name')['Age'].first().reset_index(), on='Name')
pitcher_proj['IP'] = pitching.groupby('Name')['IP'].sum().values

In [12]:
def calculate_hitting_metrics(group):
    divisor = 0
    for year, group_data in group.groupby('Year'):
        divisor += hitting_weights[year]
        
    result = {}
    for metric in ['K/PA', 'BB/PA', 'HR/PA', '1B/PA', '2B/PA', '3B/PA', 'SB/PA', 'HBP/PA', 'R/PA', 'AB/PA', 'SF/PA', 'CS/PA']:
        result[metric] = (group[metric] * group.groupby('Year')['Year'].transform(lambda x: int(x.iloc[0]) if not isinstance(x.iloc[0], int) else x.iloc[0]).map(hitting_weights) / divisor).sum()

    return pd.Series(result)

hitter_proj = hitting.groupby('Name', group_keys=False).apply(calculate_hitting_metrics).reset_index()
hitter_proj = hitter_proj.merge(hitting.groupby('Name')['Age'].first().reset_index(), on='Name')
hitter_proj['PA'] = hitting.groupby('Name')['PA'].sum().values

In [13]:
avg_Kp = pitcher_proj['K/In'].mean()
avg_BBp = pitcher_proj['BB/In'].mean()
avg_HRp = pitcher_proj['HR/In'].mean()
avg_Wp = pitcher_proj['W/In'].mean()
avg_Lp = pitcher_proj['L/In'].mean()
avg_Sp = pitcher_proj['S/In'].mean()
avg_ERp = pitcher_proj['ER/In'].mean()
avg_Rp = pitcher_proj['R/In'].mean()
avg_Gp = pitcher_proj['G/In'].mean()
avg_GSp = pitcher_proj['GS/In'].mean()

mask = pitcher_proj['IP'] < 100

pitcher_proj.loc[mask, 'K/In'] = (pitcher_proj.loc[mask, 'K/In'] * pitcher_proj.loc[mask, 'IP'] +
                                (100 - pitcher_proj.loc[mask, 'IP']) * avg_Kp) / 100

pitcher_proj.loc[mask, 'BB/In'] = (pitcher_proj.loc[mask, 'BB/In'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_BBp) / 100

pitcher_proj.loc[mask, 'HR/In'] = (pitcher_proj.loc[mask, 'HR/In'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_HRp) / 100

pitcher_proj.loc[mask, 'W/In'] = (pitcher_proj.loc[mask, 'W/In'] * pitcher_proj.loc[mask, 'IP'] +
                                (100 - pitcher_proj.loc[mask, 'IP']) * avg_Wp) / 100

pitcher_proj.loc[mask, 'L/In'] = (pitcher_proj.loc[mask, 'L/In'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_Lp) / 100

pitcher_proj.loc[mask, 'S/In'] = (pitcher_proj.loc[mask, 'S/In'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_Sp) / 100

pitcher_proj.loc[mask, 'ER/In'] = (pitcher_proj.loc[mask, 'ER/In'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_ERp) / 100

pitcher_proj.loc[mask, 'R/In'] = (pitcher_proj.loc[mask, 'R/In'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_Rp) / 100

pitcher_proj.loc[mask, 'G/In'] = (pitcher_proj.loc[mask, 'G/In'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_Gp) / 100

pitcher_proj.loc[mask, 'GS/In'] = (pitcher_proj.loc[mask, 'GS/In'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_GSp) / 100

pitcher_proj['Reliability'] = 1 - (100 - pitcher_proj['IP']).clip(0) / 100

In [14]:
avg_Kp = hitter_proj['K/PA'].mean()
avg_BBp = hitter_proj['BB/PA'].mean()
avg_HRp = hitter_proj['HR/PA'].mean()
avg_s = hitter_proj['1B/PA'].mean()
avg_d = hitter_proj['2B/PA'].mean()
avg_t = hitter_proj['3B/PA'].mean()
avg_sb = hitter_proj['SB/PA'].mean()
avg_r = hitter_proj['R/PA'].mean()
avg_hbp = hitter_proj['HBP/PA'].mean()
avg_ab = hitter_proj['AB/PA'].mean()
avg_sf = hitter_proj['SF/PA'].mean()
avg_cs = hitter_proj['CS/PA'].mean()

mask = hitter_proj['PA'] < 400

hitter_proj.loc[mask, 'K/PA'] = (hitter_proj.loc[mask, 'K/PA'] * hitter_proj.loc[mask, 'PA'] +
                                (400 - hitter_proj.loc[mask, 'PA']) * avg_Kp) / 400

hitter_proj.loc[mask, 'BB/PA'] = (hitter_proj.loc[mask, 'BB/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_BBp) / 400

hitter_proj.loc[mask, 'HR/PA'] = (hitter_proj.loc[mask, 'HR/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_HRp) / 400

hitter_proj.loc[mask, '1B/PA'] = (hitter_proj.loc[mask, '1B/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_s) / 400

hitter_proj.loc[mask, '2B/PA'] = (hitter_proj.loc[mask, '2B/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_d) / 400

hitter_proj.loc[mask, '3B/PA'] = (hitter_proj.loc[mask, '3B/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_t) / 400

hitter_proj.loc[mask, 'SB/PA'] = (hitter_proj.loc[mask, 'SB/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_sb) / 400

hitter_proj.loc[mask, 'R/PA'] = (hitter_proj.loc[mask, 'R/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_r) / 400

hitter_proj.loc[mask, 'HBP/PA'] = (hitter_proj.loc[mask, 'HBP/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_hbp) / 400

hitter_proj.loc[mask, 'AB/PA'] = (hitter_proj.loc[mask, 'AB/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_ab) / 400

hitter_proj.loc[mask, 'SF/PA'] = (hitter_proj.loc[mask, 'SF/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_sf) / 400

hitter_proj.loc[mask, 'CS/PA'] = (hitter_proj.loc[mask, 'CS/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_cs) / 400

hitter_proj['Reliability'] = 1 - (400 - hitter_proj['PA']).clip(0) / 400

In [15]:
projections_24 = pitching_23[['Name', 'Team']].merge(pitcher_proj, on='Name', how='inner')
projections_hit_24 = hitting_23[['Name', 'Team']].merge(hitter_proj, on='Name', how='inner')

In [16]:
for p in projections_24.Name.unique():
    p_data = pitching_22[pitching_22['Name'] == p]
    ip_23 = pitching_23[pitching_23['Name'] == p].IP.values[0]
    if len(p_data) == 0:
        if ip_23 <= pitching_23.IP.mean():
            proj_ip = .5 * ip_23 + 15
        else:
            proj_ip = .5 * ip_23 + 30
    else: 
        if ip_23 <= pitching_23.IP.mean():
            proj_ip = .5 * ip_23 + .1 * p_data['IP'].values[0] + 15
        else:
            proj_ip = .5 * ip_23 + .1 * p_data['IP'].values[0] + 30
    projections_24.loc[projections_24['Name'] == p, 'IP'] = proj_ip

In [17]:
for h in projections_hit_24['Name'].unique():
    h_data = hitting_22[hitting_22['Name'] == h]
    pa_23 = hitting_23[hitting_23['Name'] == h]['PA'].values[0]
    if len(h_data) == 0:
        proj_pa = 0.5 * pa_23 + 100
    else:
        proj_pa = 0.5 * pa_23 + 0.1 * h_data['PA'].values[0] + 100
    projections_hit_24.loc[projections_hit_24['Name'] == h, 'PA'] = proj_pa

C:\Users\kellyjc\AppData\Local\Temp\ipykernel_10504\317420038.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '354.9' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  projections_hit_24.loc[projections_hit_24['Name'] == h, 'PA'] = proj_pa


In [18]:
projections_24['Age'] = pd.to_numeric(projections_24['Age'], errors='coerce')
projections_24['Age'] += 1
projections_24.loc[projections_24['Age'] >= 29, 'AgeAdj'] = 1 / (1 + (((projections_24['Age'] - 29) - 29) * 0.003))
projections_24.loc[projections_24['Age'] < 29, 'AgeAdj'] = 1 + ((29 - (projections_24['Age'] - 29)) * 0.006)

In [19]:
projections_hit_24['Age'] = pd.to_numeric(projections_hit_24['Age'], errors='coerce')
projections_hit_24['Age'] += 1
projections_hit_24.loc[projections_hit_24['Age'] >= 29, 'AgeAdj'] = 1 / (1 + (((projections_hit_24['Age'] - 29) - 29) * 0.003))
projections_hit_24.loc[projections_hit_24['Age'] < 29, 'AgeAdj'] = 1 + ((29 - (projections_hit_24['Age'] - 29)) * 0.006)

In [20]:
projections_24['IP'] *= projections_24['AgeAdj']
projections_hit_24['PA'] *= projections_hit_24['AgeAdj']

In [21]:
projections_24 = projections_24.dropna(subset=['IP'])
projections_hit_24 = projections_hit_24.dropna(subset=['PA'])

In [22]:
projections_24['K'] = projections_24['K/In'] * projections_24['IP']
projections_24['BB'] = projections_24['BB/In'] * projections_24['IP']
projections_24['HR'] = projections_24['HR/In'] * projections_24['IP']
projections_24['W'] = projections_24['W/In'] * projections_24['IP']
projections_24['L'] = projections_24['L/In'] * projections_24['IP']
projections_24['S'] = projections_24['S/In'] * projections_24['IP']
projections_24['ER'] = projections_24['ER/In'] * projections_24['IP']
projections_24['R'] = projections_24['R/In'] * projections_24['IP']
projections_24['G'] = projections_24['G/In'] * projections_24['IP']
projections_24['GS'] = projections_24['GS/In'] * projections_24['IP']

projections_24['K'] = projections_24['K'].apply(math.floor)
projections_24['BB'] = projections_24['BB'].apply(math.floor)
projections_24['HR'] = projections_24['HR'].apply(math.floor)
projections_24['W'] = projections_24['W'].apply(math.floor)
projections_24['L'] = projections_24['L'].apply(math.floor)
projections_24['S'] = projections_24['S'].apply(math.floor)
projections_24['ER'] = projections_24['ER'].apply(math.floor)
projections_24['R'] = projections_24['R'].apply(math.floor)
projections_24['G'] = projections_24['G'].apply(math.floor)
projections_24['GS'] = projections_24['GS'].apply(math.floor)

In [23]:
projections_hit_24['SB'] = projections_hit_24['SB/PA'] * projections_hit_24['PA']
projections_hit_24['HR'] = projections_hit_24['HR/PA'] * projections_hit_24['PA']
projections_hit_24['K'] = projections_hit_24['K/PA'] * projections_hit_24['PA']
projections_hit_24['BB'] = projections_hit_24['BB/PA'] * projections_hit_24['PA']
projections_hit_24['1B'] = projections_hit_24['1B/PA'] * projections_hit_24['PA']
projections_hit_24['2B'] = projections_hit_24['2B/PA'] * projections_hit_24['PA']
projections_hit_24['3B'] = projections_hit_24['3B/PA'] * projections_hit_24['PA']
projections_hit_24['SO'] = projections_hit_24['K/PA'] * projections_hit_24['PA']
projections_hit_24['R'] = projections_hit_24['R/PA'] * projections_hit_24['PA']
projections_hit_24['SB'] = projections_hit_24['SB/PA'] * projections_hit_24['PA']
projections_hit_24['HBP'] = projections_hit_24['HBP/PA'] * projections_hit_24['PA']
projections_hit_24['SF'] = projections_hit_24['SF/PA'] * projections_hit_24['PA']
projections_hit_24['AB'] = projections_hit_24['AB/PA'] * projections_hit_24['PA']
projections_hit_24['CS'] = projections_hit_24['CS/PA'] * projections_hit_24['PA']

projections_hit_24['K'] = projections_hit_24['K'].apply(math.floor)
projections_hit_24['BB'] = projections_hit_24['BB'].apply(math.floor)
projections_hit_24['HR'] = projections_hit_24['HR'].apply(math.floor)
projections_hit_24['SB'] = projections_hit_24['SB'].apply(math.floor)
projections_hit_24['1B'] = projections_hit_24['1B'].apply(math.floor)
projections_hit_24['2B'] = projections_hit_24['2B'].apply(math.floor)
projections_hit_24['3B'] = projections_hit_24['3B'].apply(math.floor)
projections_hit_24['SO'] = projections_hit_24['SO'].apply(math.floor)
projections_hit_24['R'] = projections_hit_24['R'].apply(math.floor)
projections_hit_24['SB'] = projections_hit_24['SB'].apply(math.floor)
projections_hit_24['HBP'] = projections_hit_24['HBP'].apply(math.floor)
projections_hit_24['AB'] = projections_hit_24['AB'].apply(math.floor)
projections_hit_24['SF'] = projections_hit_24['SF'].apply(math.floor)
projections_hit_24['CS'] = projections_hit_24['CS'].apply(math.floor)

In [24]:
woba_vals = {'BB': 0.696, 'HBP': 0.726, '1B': 0.883, '2B': 1.244, '3B': 1.569, 'HR': 2.004, 'SB': .200, 'CS': .422}

def calculate_woba(df):
    woba = (
        (woba_vals['BB'] * df['BB'] +
        woba_vals['1B'] * df['1B'] +
        woba_vals['2B'] * df['2B'] +
        woba_vals['3B'] * df['3B'] +
        woba_vals['HR'] * df['HR'])  
        / (df['AB'] + df['BB'] + df['SF'] + df['HBP']))
    return woba

def calculate_woba_war(df):
    woba = (
        (woba_vals['BB'] * df['BB'] +
        woba_vals['1B'] * df['1B'] +
        woba_vals['2B'] * df['2B'] +
        woba_vals['3B'] * df['3B'] +
        woba_vals['HR'] * df['HR'] +
        woba_vals['CS'] * df['CS'] +
        woba_vals['SB'] * df['SB'])  
        / (df['AB'] + df['BB'] + df['SF']))
    return woba

def calculate_babip(df):
    babip = (
        (df['H'] - df['HR']) / (df['AB'] - df['K'] - df['HR'] + df['SF']))
    return babip

In [25]:
projections_hit_24['H'] = projections_hit_24['1B'] + projections_hit_24['2B'] + projections_hit_24['3B'] + projections_hit_24['HR']

projections_hit_24['wOBA'] = calculate_woba(projections_hit_24)
projections_hit_24['wOBA_WAR'] = calculate_woba_war(projections_hit_24) 
projections_hit_24['BAbip'] = calculate_babip(projections_hit_24)
projections_hit_24['OBP'] = (projections_hit_24['H'] + projections_hit_24['HBP'] + projections_hit_24['BB']) / projections_hit_24['PA']
projections_hit_24['BA'] = projections_hit_24['H'] / (projections_hit_24['AB'])
projections_hit_24['SLG'] = (projections_hit_24['1B'] * 1 + projections_hit_24['2B'] * 2 + projections_hit_24['3B'] * 3 + projections_hit_24['HR'] * 4) / (projections_hit_24['AB'])

In [26]:
def calculate_fip(df):
    f_constant = df.ERA.mean() - (((13 * df.HR.sum()) + (3 * (df.BB.sum())) - (2 * df.K.sum())) / df.IP.sum())
    return f_constant + (((13 * df.HR) + (3 * df.BB) - (2 * df.K)) / df.IP)

projections_24['ERA'] = 9 * projections_24['ER'] / projections_24['IP']
projections_24['FIP'] = calculate_fip(projections_24)
projections_24['BB/9'] = (projections_24['BB'] / projections_24['IP']) * 9
projections_24['RA9'] = (projections_24['R'] / projections_24['IP']) * 9
projections_24['K/9'] = (projections_24['K'] / projections_24['IP']) * 9
projections_24['K/BB'] = projections_24['K']  / projections_24['BB']

In [27]:
projections_24 = projections_24[['Name', 'Age', 'Team', 'IP', 'G', 'GS', 'W', 'L', 'S', 'K', 'BB', 'HR', 'R', 'ERA', 'FIP', 'BB/9', 'K/9', 'K/BB', 'RA9', 'Reliability']]

In [28]:
projections_hit_24 = projections_hit_24[['Name', 'Age', 'Team', 'PA', '1B', '2B', '3B', 'HR', 'R', 'SO', 'SB', 'BB', 'HBP', 'wOBA', 'BA', 'SLG', 'OBP', 'BAbip', 'wOBA_WAR', 'Reliability']]

In [29]:
projections_24.to_csv('../Frontier-League-Savant/csvs/2024 Pitcher Projections.csv')

In [30]:
projections_hit_24.to_csv('../Frontier-League-Savant/csvs/2024 Hitter Projections.csv')